In [1]:
import numpy as np
import pandas as pd

將17個data合併，把data中'Pressure(hpa)'>1050的資料改成1050，'Humidity(%)'>100的資料改成100，並存成新的檔案datanew，捨去資料中'WindSpeed(m/s)'欄位

In [2]:
#讀取檔案
datanew = pd.read_csv('./data/L_data/datanew.csv')
datanew.shape


(1375028, 8)

In [3]:
#不要datanew中'WindSpeed(m/s)'欄位
datanew.drop(['WindSpeed(m/s)'],axis=1,inplace=True)
datanew.columns

Index(['LocationCode', 'DateTime', 'Pressure(hpa)', 'Temperature(°C)',
       'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')

定義'Power'反歸一化函數，並設定歸一化條件

In [4]:
# 預先計算最大和最小值
max_power = datanew['Power(mW)'].max()
min_power = datanew['Power(mW)'].min()
range_power = max_power - min_power
# 預先計算最大和最小值
print(range_power)
print(min_power)
print(max_power)

# 定義函數，使用預計算的最大值和最小值
def inverse_transform_power(value):
    return value * range_power + min_power

# 定義函數，使用預計算的最大值和最小值
def inverse_transform_power(value):
    return value * 2626.48 + 0.0

2626.48
0.0
2626.48


In [10]:
#對datanew中的'Pressure(hpa)','Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'做歸一化
name=""
if name=="_maxmin":
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    datanew[['Pressure(hpa)','Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']] = scaler.fit_transform(datanew[['Pressure(hpa)','Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']])
    datanew

去掉07:00以前和17:00以後的資料，並增加'Date'、'Time'、'TimeMin'欄位

In [5]:
#將data_1按照DateTime排序
data_1 = datanew.sort_values('DateTime')
data_1.head(3)

,LocationCode,DateTime,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
1278386,17,2024-01-01 06:29:01.000,1022.91,17.9,63.1,22.50,0.00
1278387,17,2024-01-01 06:30:01.000,1023.01,17.9,63.1,29.17,0.00
1278388,17,2024-01-01 06:31:01.000,1023.06,17.9,63.2,35.00,0.01


In [6]:
# 將 DateTime 欄位轉換為 datetime 格式
data_1['DateTime'] = pd.to_datetime(data_1['DateTime'])

# 過濾條件：僅保留 07:00:00 - 17:00:00 的資料
filtered_data = data_1[(data_1['DateTime'].dt.time >= pd.to_datetime('07:00:00').time()) &
                       (data_1['DateTime'].dt.time <= pd.to_datetime('16:59:59').time())]

#對filtered_data創建新欄位"Date"，並將DateTime欄位中的日期部分取出存入Date欄位
filtered_data['Date'] = filtered_data['DateTime'].dt.date

#對filtered_data創建新欄位"Time"，並將DateTime欄位中的時間部分取出存入Time欄位
filtered_data['Time'] = filtered_data['DateTime'].dt.time

#對filtered_data創建新欄位"TimeMin"，並將DateTime欄位中的時間部分(時分)取出存入TimeMin欄位
filtered_data['TimeMin'] = filtered_data['DateTime'].dt.strftime('%H:%M')
filtered_data.head(3)

/tmp/ipykernel_3038116/1014523967.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Date'] = filtered_data['DateTime'].dt.date
/tmp/ipykernel_3038116/1014523967.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Time'] = filtered_data['DateTime'].dt.time
/tmp/ipykernel_3038116/1014523967.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,LocationCode,DateTime,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),Date,Time,TimeMin
1278417,17,2024-01-01 07:00:01,1023.19,18.0,64.4,617.50,0.13,2024-01-01,07:00:01,07:00
29,1,2024-01-01 07:00:07,1016.69,17.5,86.9,415.83,0.07,2024-01-01,07:00:07,07:00
1278418,17,2024-01-01 07:01:01,1023.18,18.0,64.7,758.33,0.14,2024-01-01,07:01:01,07:01


計算各TimeMin變項平均值，並將缺少的各'LocationCode', 'Date'之'TimeMin'補足

In [7]:
# 按 LocationCode, Date, 和 TimeMin 分組，並計算各變項的平均值
grouped_result = filtered_data.groupby(['LocationCode', 'Date', 'TimeMin']).agg(
    {
        'Pressure(hpa)': 'mean',
        'Temperature(°C)': 'mean',
        'Humidity(%)': 'mean',
        'Sunlight(Lux)': 'mean',
        'Power(mW)': 'mean'
    }
).reset_index()

grouped_result.head(3)

,LocationCode,Date,TimeMin,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01,07:00,1016.69,17.5,86.9,415.83,0.07
1,1,2024-01-01,07:01,1016.71,17.5,87.0,429.17,0.08
2,1,2024-01-01,07:02,1016.69,17.5,87.0,434.17,0.08


In [8]:
#看grouped_result中TimeMin有哪些值
grouped_result['TimeMin'].unique()

array(['07:00', '07:01', '07:02', '07:03', '07:04', '07:05', '07:06',
       '07:07', '07:08', '07:09', '07:10', '07:11', '07:12', '07:13',
       '07:14', '07:15', '07:16', '07:17', '07:18', '07:19', '07:20',
       '07:21', '07:22', '07:23', '07:24', '07:25', '07:26', '07:27',
       '07:28', '07:29', '07:30', '07:31', '07:32', '07:33', '07:34',
       '07:35', '07:36', '07:37', '07:38', '07:39', '07:40', '07:41',
       '07:42', '07:43', '07:44', '07:45', '07:46', '07:47', '07:48',
       '07:49', '07:50', '07:51', '07:52', '07:53', '07:54', '07:55',
       '07:56', '07:57', '07:58', '07:59', '08:00', '08:01', '08:02',
       '08:03', '08:04', '08:05', '08:06', '08:07', '08:08', '08:09',
       '08:10', '08:11', '08:12', '08:13', '08:14', '08:15', '08:16',
       '08:17', '08:18', '08:19', '08:20', '08:21', '08:22', '08:23',
       '08:24', '08:25', '08:26', '08:27', '08:28', '08:29', '08:30',
       '08:31', '08:32', '08:33', '08:34', '08:35', '08:36', '08:37',
       '08:38', '08:

In [11]:
# 定義時間區間，從 07:00 到 08:59，每隔1分鐘
time_range = pd.date_range("07:00", "08:59", freq="1T").strftime('%H:%M')

# 獲取唯一的 LocationCode 和 Date 組合
unique_location_dates = grouped_result[['LocationCode', 'Date']].drop_duplicates()

# 擴展 DataFrame，以包含每個 LocationCode 和 Date 的完整 TimeMin
expanded_result = pd.DataFrame(
    [
        {'LocationCode': loc, 'Date': date, 'TimeMin': time}
        for loc, date in unique_location_dates.itertuples(index=False)
        for time in time_range
    ]
)

# 與 grouped_result 合併，補齊缺少的時間段，並對缺失數據填入 NaN
final_result = pd.merge(expanded_result, grouped_result, on=['LocationCode', 'Date', 'TimeMin'], how='left')

final_result.head(3)

#將final_result存成csv檔
final_result.to_csv(f'./data/preprocess_data/TimeMinData{name}.csv',index=False)

/tmp/ipykernel_3038116/1931572576.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_range = pd.date_range("07:00", "08:59", freq="1T").strftime('%H:%M')


計算每個 LocationCode 和 Date 的缺失值行數，將當天120個時段都缺失的刪除，並將剩下的對每個 LocationCode 和 Date 組合進行缺失值插補

In [12]:
final_result_1 = final_result.copy()

In [13]:
# 計算每個 LocationCode 和 Date 的缺失值行數
missing_count_per_group = final_result_1.groupby(['LocationCode', 'Date']).apply(
    lambda group: group.isnull().any(axis=1).sum()  # 計算該組中至少有一個缺失值的行數
).reset_index(name='MissingRowsCount')

# 找出需要刪除的組合（缺失值行數等於 120 的組合）
groups_to_delete = missing_count_per_group[missing_count_per_group['MissingRowsCount'] == 120][['LocationCode', 'Date']]

# 過濾原始 DataFrame，刪除這些組合的資料
filtered_result = final_result_1.merge(groups_to_delete, on=['LocationCode', 'Date'], how='outer', indicator=True)
filtered_result = filtered_result[filtered_result['_merge'] == 'left_only'].drop(columns=['_merge'])

filtered_result.shape


/tmp/ipykernel_3038116/1614795101.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_count_per_group = final_result_1.groupby(['LocationCode', 'Date']).apply(


(233760, 8)

In [14]:
# 對每個 LocationCode 和 Date 組合進行缺失值插補
interpolated_result = (
    filtered_result
    .groupby(['LocationCode', 'Date'], group_keys=False)  # 按 LocationCode 和 Date 分組
    .apply(lambda group: group.sort_values(by='TimeMin').interpolate(method='linear', limit_direction='both'))
)

interpolated_result.head(3)


/tmp/ipykernel_3038116/4179453808.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.sort_values(by='TimeMin').interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_3038116/4179453808.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.sort_values(by='TimeMin').interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_3038116/4179453808.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.sort_values(by='TimeMin').interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_3038116/4179453808.py:5: Fut

,LocationCode,Date,TimeMin,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01,07:00,1016.69,17.5,86.9,415.83,0.07
1,1,2024-01-01,07:01,1016.71,17.5,87.0,429.17,0.08
2,1,2024-01-01,07:02,1016.69,17.5,87.0,434.17,0.08


用interpolated_result(插補完成的資料)，針對每一個LocationCode和Date，將各TimeMin的Pressure(hpa)、Temperature(°C)、Humidity(%)、Sunlight(Lux)、Power(mW)值，轉成在欄位的地方

In [15]:
# 確定需要轉換的欄位
variables = ['Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']

# 將每個變數的數據轉換為寬格式
reshaped_data = []
for variable in variables:
    reshaped = interpolated_result.pivot_table(
        index=['LocationCode', 'Date'],  # 使用 LocationCode 和 Date 作為索引
        columns='TimeMin',               # 時間段作為列名
        values=variable                  # 變數作為值
    )
    
    # 將列名調整為 "時間_變數" 的形式
    reshaped.columns = [f"{time}_{variable}" for time in reshaped.columns]
    reshaped_data.append(reshaped)

# 合併所有變數的結果
final_reshaped_result = pd.concat(reshaped_data, axis=1).reset_index()

# 按時間和變數排序列名
# 提取需要排序的欄位
columns_to_sort = final_reshaped_result.columns.difference(['LocationCode', 'Date'])

# 自定義排序邏輯
def custom_sort(column_name):
    time, variable = column_name.split('_', 1)
    variable_order = {var: i for i, var in enumerate(variables)}  # 定義變數的固定順序
    return (time, variable_order[variable])

# 按時間和變數固定順序排序
sorted_columns = sorted(columns_to_sort, key=custom_sort)

# 重新排列列的順序
final_reshaped_result = final_reshaped_result[['LocationCode', 'Date'] + sorted_columns]

final_reshaped_result.head(3)

final_reshaped_result.to_csv(f'./data/preprocess_data/TimeMinData_Reshaped{name}.csv',index=False)

用TimeMinData_Reshaped{name}和AllVariablesData_na_testset_1來重新整理測試集

In [16]:
test_data = pd.read_csv('./data/preprocess_data/AllVariablesData_na_testset_1.csv')
test_data.head(3)

,LocationCode,Date_Day1,Date_Day2,07:00_1_Pressure(hpa),07:00_1_Temperature(°C),07:00_1_Humidity(%),07:00_1_Sunlight(Lux),07:00_1_Power(mW),07:10_1_Pressure(hpa),07:10_1_Temperature(°C),...,08:40_2_Pressure(hpa),08:40_2_Temperature(°C),08:40_2_Humidity(%),08:40_2_Sunlight(Lux),08:40_2_Power(mW),08:50_2_Pressure(hpa),08:50_2_Temperature(°C),08:50_2_Humidity(%),08:50_2_Sunlight(Lux),08:50_2_Power(mW)
0,1,2024-01-05,2024-01-06,1015.319,12.72,100.0,4887.751,16.565,1015.384,14.38,...,1015.805000,26.300000,57.190000,56151.271000,870.177000,1015.774,27.46,52.75,66608.359,1106.220
1,1,2024-01-10,2024-01-11,1010.490,18.11,100.0,961.751,0.424,1010.612,18.21,...,1016.577778,17.388889,85.088889,11982.871111,32.345556,1016.581,17.80,82.83,13097.500,38.685
2,1,2024-01-13,2024-01-14,1015.253,14.01,100.0,2210.333,1.172,1015.390,14.27,...,1017.535000,19.870000,89.560000,31820.665000,221.674000,1017.602,22.13,78.94,49692.799,578.049


In [17]:
#把test_data第四欄位到最後一欄位刪除
test_data.drop(test_data.columns[3:],axis=1,inplace=True)
test_data.head(3)

,LocationCode,Date_Day1,Date_Day2
0,1,2024-01-05,2024-01-06
1,1,2024-01-10,2024-01-11
2,1,2024-01-13,2024-01-14


In [19]:
#將test_data和final_reshaped_result左合併，依照train_data的LocationCode、Date_Day2對應final_reshaped_result的LocationCode、Date
final_data_test = pd.merge(test_data, final_reshaped_result, left_on=['LocationCode','Date_Day2'], right_on=['LocationCode','Date'], how='left')

#將final_data的Date欄位刪除
final_data_test.drop(['Date'],axis=1,inplace=True)
final_data_test.head(3)

final_data_test.to_csv('./data/preprocess_data/AllVariablesData_oneday_test.csv',index=False)

In [28]:
#列出final_data_test中所有欄位(tolist)
final_data_test.columns.tolist()
print(final_data_test.columns.tolist())

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_Pressure(hpa)', '07:00_Temperature(°C)', '07:00_Humidity(%)', '07:00_Sunlight(Lux)', '07:00_Power(mW)', '07:01_Pressure(hpa)', '07:01_Temperature(°C)', '07:01_Humidity(%)', '07:01_Sunlight(Lux)', '07:01_Power(mW)', '07:02_Pressure(hpa)', '07:02_Temperature(°C)', '07:02_Humidity(%)', '07:02_Sunlight(Lux)', '07:02_Power(mW)', '07:03_Pressure(hpa)', '07:03_Temperature(°C)', '07:03_Humidity(%)', '07:03_Sunlight(Lux)', '07:03_Power(mW)', '07:04_Pressure(hpa)', '07:04_Temperature(°C)', '07:04_Humidity(%)', '07:04_Sunlight(Lux)', '07:04_Power(mW)', '07:05_Pressure(hpa)', '07:05_Temperature(°C)', '07:05_Humidity(%)', '07:05_Sunlight(Lux)', '07:05_Power(mW)', '07:06_Pressure(hpa)', '07:06_Temperature(°C)', '07:06_Humidity(%)', '07:06_Sunlight(Lux)', '07:06_Power(mW)', '07:07_Pressure(hpa)', '07:07_Temperature(°C)', '07:07_Humidity(%)', '07:07_Sunlight(Lux)', '07:07_Power(mW)', '07:08_Pressure(hpa)', '07:08_Temperature(°C)', '07:08_Humidity(%)', 

用TimeMinData_Reshaped{name}和AllVariablesData來重新整理訓練集

In [21]:
train_data = pd.read_csv('./data/preprocess_data/AllVariablesData.csv')
train_data.head(3)

,LocationCode,Date_Day1,Date_Day2,07:00_1_Pressure(hpa),07:00_1_Temperature(°C),07:00_1_Humidity(%),07:00_1_Sunlight(Lux),07:00_1_Power(mW),07:10_1_Pressure(hpa),07:10_1_Temperature(°C),...,16:40_2_Pressure(hpa),16:40_2_Temperature(°C),16:40_2_Humidity(%),16:40_2_Sunlight(Lux),16:40_2_Power(mW),16:50_2_Pressure(hpa),16:50_2_Temperature(°C),16:50_2_Humidity(%),16:50_2_Sunlight(Lux),16:50_2_Power(mW)
0,1,2024-01-01,2024-01-02,1016.730,17.50,86.99,470.834,0.083,1016.774,17.57,...,1010.348,23.05,62.34,2538.582,1.72,1010.415,22.65,63.57,1448.001,0.578
1,1,2024-01-02,2024-01-03,1014.651,12.99,100.00,4352.583,8.296,1014.810,14.44,...,1012.824,17.81,89.19,261.249,0.03,1013.030,17.71,89.97,186.417,0.016
2,1,2024-01-03,2024-01-04,1011.794,15.44,100.00,1389.418,0.557,1011.841,15.65,...,1014.575,18.78,69.59,1704.335,0.82,1014.761,18.53,70.75,1164.249,0.405


In [22]:
#將train_data和final_reshaped_result左合併，依照train_data的LocationCode、Date_Day2對應final_reshaped_result的LocationCode、Date
final_data = pd.merge(train_data, final_reshaped_result, left_on=['LocationCode','Date_Day2'], right_on=['LocationCode','Date'], how='left')
final_data.head(3)

,LocationCode,Date_Day1,Date_Day2,07:00_1_Pressure(hpa),07:00_1_Temperature(°C),07:00_1_Humidity(%),07:00_1_Sunlight(Lux),07:00_1_Power(mW),07:10_1_Pressure(hpa),07:10_1_Temperature(°C),...,08:58_Pressure(hpa),08:58_Temperature(°C),08:58_Humidity(%),08:58_Sunlight(Lux),08:58_Power(mW),08:59_Pressure(hpa),08:59_Temperature(°C),08:59_Humidity(%),08:59_Sunlight(Lux),08:59_Power(mW)
0,1,2024-01-01,2024-01-02,1016.730,17.50,86.99,470.834,0.083,1016.774,17.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-01-02,2024-01-03,1014.651,12.99,100.00,4352.583,8.296,1014.810,14.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2024-01-03,2024-01-04,1011.794,15.44,100.00,1389.418,0.557,1011.841,15.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#將final_data的4~363欄位刪除
final_data.drop(final_data.iloc[:, 3:363], axis=1, inplace=True)

#將final_data的Date欄位刪除
final_data.drop(['Date'],axis=1,inplace=True)
final_data.head(3)

,LocationCode,Date_Day1,Date_Day2,09:00_2_Pressure(hpa),09:00_2_Temperature(°C),09:00_2_Humidity(%),09:00_2_Sunlight(Lux),09:00_2_Power(mW),09:10_2_Pressure(hpa),09:10_2_Temperature(°C),...,08:58_Pressure(hpa),08:58_Temperature(°C),08:58_Humidity(%),08:58_Sunlight(Lux),08:58_Power(mW),08:59_Pressure(hpa),08:59_Temperature(°C),08:59_Humidity(%),08:59_Sunlight(Lux),08:59_Power(mW)
0,1,2024-01-01,2024-01-02,1014.782,31.01,47.78,70884.743,1277.553,1014.696,32.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-01-02,2024-01-03,1012.623,23.43,77.24,27053.085,158.704,1012.585,24.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2024-01-03,2024-01-04,1017.352,17.66,86.06,10590.000,25.631,1017.364,17.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# 獲取列名稱列表
columns = final_data.columns.tolist()

# 提取第1到第3欄位的列名稱
cols_to_keep_start = columns[:3]

# 提取第4到第243欄位的列名稱
cols_to_keep_end = columns[3:243]

# 提取第244欄位到最後一欄的列名稱
cols_to_move_2 = columns[243:]  # 第617欄的索引為616

# 重新排列列順序
new_column_order = cols_to_keep_start + cols_to_move_2  + cols_to_keep_end

# 重新排列數據
data_after = final_data[new_column_order]
data_after.head(3)

data_after.to_csv('./data/preprocess_data/AllVariablesData_oneday.csv',index=False)

In [29]:
#列出final_data_test中所有欄位(tolist)
data_after.columns.tolist()
print(data_after.columns.tolist())

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_Pressure(hpa)', '07:00_Temperature(°C)', '07:00_Humidity(%)', '07:00_Sunlight(Lux)', '07:00_Power(mW)', '07:01_Pressure(hpa)', '07:01_Temperature(°C)', '07:01_Humidity(%)', '07:01_Sunlight(Lux)', '07:01_Power(mW)', '07:02_Pressure(hpa)', '07:02_Temperature(°C)', '07:02_Humidity(%)', '07:02_Sunlight(Lux)', '07:02_Power(mW)', '07:03_Pressure(hpa)', '07:03_Temperature(°C)', '07:03_Humidity(%)', '07:03_Sunlight(Lux)', '07:03_Power(mW)', '07:04_Pressure(hpa)', '07:04_Temperature(°C)', '07:04_Humidity(%)', '07:04_Sunlight(Lux)', '07:04_Power(mW)', '07:05_Pressure(hpa)', '07:05_Temperature(°C)', '07:05_Humidity(%)', '07:05_Sunlight(Lux)', '07:05_Power(mW)', '07:06_Pressure(hpa)', '07:06_Temperature(°C)', '07:06_Humidity(%)', '07:06_Sunlight(Lux)', '07:06_Power(mW)', '07:07_Pressure(hpa)', '07:07_Temperature(°C)', '07:07_Humidity(%)', '07:07_Sunlight(Lux)', '07:07_Power(mW)', '07:08_Pressure(hpa)', '07:08_Temperature(°C)', '07:08_Humidity(%)', 